In [1]:
from utils import TestResults,TrainSplit
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import log_loss,f1_score
import pandas as pd
import numpy as np
import optuna
import gc

In [2]:
X_train, X_test, y_train, y_test = TrainSplit().get_split()

Loading the data
Categorial Feature Imputed


NUM IMPUTE: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 45.30it/s]


Spliting the Data based on time
X_train:(3387880, 80), X_test:(97972, 80) , y_train:(3387880, 2) , y_test:(97972, 2)


In [3]:
cat_feat = TrainSplit.CATEGORIES + TrainSplit.BINARY
num_feat = TrainSplit.NUMERICAL
all_feat = cat_feat + num_feat

In [4]:
catboost_params = {'depth': 5, 'iterations': 633, 'learning_rate': 0.11345058095313602, 
                   'l2_leaf_reg': 5.249467944690077, 'boosting_type': 'Ordered', 
                   'task_type': 'GPU', 'silent': True}

In [5]:
xgboost_params = {'max_depth': 7, 'learning_rate': 0.05198517520669243, 'n_estimators': 606,
                   'tree_method':'gpu_hist','objective':'binary:logistic'}

In [6]:
xgb_model = XGBClassifier(**xgboost_params)
xgb_model.fit(X_train[all_feat],y_train[TrainSplit.IS_INSTALLED[0]])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05198517520669243,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=606, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [7]:
catb_model = CatBoostClassifier(**catboost_params)
catb_model.fit(X_train[all_feat],y_train[TrainSplit.IS_INSTALLED[0]])

TBB Warning: The number of workers is currently limited to 29. The request for 39 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



In [8]:
xgb_train_pred  = xgb_model.predict_proba(X_train[all_feat])[:,1]
catb_train_pred = catb_model.predict_proba(X_train[all_feat])[:,1]

In [9]:
xgb_val_pred  = xgb_model.predict_proba(X_test[all_feat])[:,1]
catb_val_pred = catb_model.predict_proba(X_test[all_feat])[:,1]

In [10]:
# Model selection
model_sel = []# zero is xgb one is cat boost
for g,x,c in zip(y_train[TrainSplit.IS_INSTALLED[0]],xgb_train_pred,catb_train_pred):
    # print(g,x,c)
    if g == 1:
        sel =  0 if x>c else 1
    elif g == 0:
        sel =  1 if x>c else 0
    model_sel.append(sel)

In [11]:
y_train_model_sel = np.array(model_sel)

In [12]:
np.unique(y_train_model_sel,return_counts=True)

(array([0, 1]), array([2207801, 1180079]))

In [13]:
model_sel = []# zero is xgb one is cat boost
for g,x,c in zip(y_test[TrainSplit.IS_INSTALLED[0]],xgb_val_pred,catb_val_pred):
    # print(g,x,c)
    if g == 1:
        sel =  0 if x>c else 1
    elif g == 0:
        sel =  1 if x>c else 0
    model_sel.append(sel)

In [14]:
len(model_sel)

97972

In [15]:
y_val_model_sel = np.array(model_sel)

In [16]:
np.unique(y_val_model_sel,return_counts=True)

(array([0, 1]), array([51800, 46172]))

In [17]:
# X_train_pred = np.vstack([xgb_train_pred,catb_train_pred]).T
# X_val_pred = np.vstack([xgb_val_pred,catb_val_pred]).T

In [18]:
# X_train_pred.shape,X_val_pred.shape

In [38]:
def objective(trial):
    params = {
        'max_depth':trial.suggest_int('max_depth',2,8),
        'learning_rate':trial.suggest_float('learning_rate',0.05,0.3),
        'n_estimators':trial.suggest_int('n_estimators',100,1000),
        'tree_method':'gpu_hist',
        'objective':'binary:logistic'
    }
    # print(use_features)
    model = XGBClassifier(**params)
    model.fit(X_train[all_feat],y_train_model_sel)
    y_pred = model.predict(X_test[all_feat])
    score = log_loss(y_val_model_sel,y_pred)
    # print(model.feature_names_in_)
    return score

In [ ]:
study_stack = optuna.create_study(direction='minimize',study_name='svc_log_loss')
study_stack.optimize(objective, n_trials=50)

[I 2023-05-25 19:05:44,923] A new study created in memory with name: svc_log_loss
[I 2023-05-25 19:06:13,859] Trial 0 finished with value: 12.31525103938559 and parameters: {'max_depth': 5, 'learning_rate': 0.12756352314624264, 'n_estimators': 781}. Best is trial 0 with value: 12.31525103938559.
[I 2023-05-25 19:06:45,262] Trial 1 finished with value: 11.438861534153258 and parameters: {'max_depth': 8, 'learning_rate': 0.2379357732135427, 'n_estimators': 579}. Best is trial 1 with value: 11.438861534153258.
[I 2023-05-25 19:07:01,893] Trial 2 finished with value: 14.266510511730209 and parameters: {'max_depth': 5, 'learning_rate': 0.07297732028383565, 'n_estimators': 353}. Best is trial 1 with value: 11.438861534153258.
[I 2023-05-25 19:07:35,978] Trial 3 finished with value: 11.465651024728773 and parameters: {'max_depth': 6, 'learning_rate': 0.21125183910889328, 'n_estimators': 817}. Best is trial 1 with value: 11.438861534153258.
[I 2023-05-25 19:07:57,179] Trial 4 finished with val

In [19]:
model_sel_param = {'max_depth': 8, 'learning_rate': 0.25475042568065964, 'n_estimators': 873
                   ,'tree_method':'gpu_hist', 'objective':'binary:logistic'}

In [20]:
model_sel = XGBClassifier(**model_sel_param)
model_sel.fit(X_train[all_feat],y_train_model_sel)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.25475042568065964,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=873, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [22]:
# import pandas as pd
test = pd.read_csv('../Data/test/000000000000.csv',sep='\t')
test['f_30'].fillna(test['f_30'].mode()[0],inplace=True)
test['f_31'].fillna(test['f_31'].mode()[0],inplace=True)
fmiss = "f_43,f_51,f_58,f_59,f_64,f_65,f_66,f_67,f_68,f_69,f_70".split(',')
for f in fmiss:
    test[f].fillna(test[f].mean(),inplace=True)

In [23]:
xgb_pred_test = xgb_model.predict_proba(test[all_feat])
catb_pred_test = catb_model.predict_proba(test[all_feat])

In [24]:
selector = model_sel.predict(test[all_feat])

In [28]:
install_prob = []
for s,x,c in zip(selector,xgb_pred_test,catb_pred_test):
    if s == 1:
        install_prob.append(c[1])
    elif s == 0:
        install_prob.append(x[1])

In [30]:
install_prob = np.array(install_prob)

In [31]:
TestResults(row_id=test['f_0'],is_click=np.random.random(install_prob.shape[0]),
            is_install=install_prob,model_name="xgb_cat_model_selection_xgb",config=model_sel_param)

Saved the test result to csv file as xgb_cat_model_selection_xgb_2023-05-25 19:28:49.160891.csv
Saved the model config to json file as xgb_cat_model_selection_xgb_2023-05-25 19:28:49.160891.json


In [3]:
TrainSplit.get_config_list()

['catboost_all_feat_2023-05-25 11:41:56.839795.json',
 'xgb_stack_all_cat_num_2023-05-25 04:08:18.733781.json',
 'xgb_cat_feat_2023-05-24 05:09:00.493398.json',
 'xgb_calibrated_logistic_2023-05-26 14:54:36.420474.json',
 'xgb_num_feat_2023-05-24 05:09:31.486438.json',
 'xgb_stack_all_cat_num_xgb_2023-05-25 06:19:35.109117.json',
 'xgb_num_cat_all_avg_2023-05-25 16:21:55.184468.json',
 'xgb_cat_model_selection_xgb_2023-05-25 19:28:49.160891.json',
 'xgb_all_feat_xgb_chain_2023-05-25 16:05:56.352515.json',
 'xgb_all_feat_2023-05-24 05:08:29.718082.json']

In [7]:
cat_output = pd.read_csv('../Data/results/catboost_all_feat_2023-05-25 11:41:56.839795.csv',sep='\t')

In [8]:
xgb_output = pd.read_csv('../Data/results/xgb_all_feat_2023-05-24 05:08:29.718082.csv',sep='\t')

In [9]:
cat_output

,RowId,is_clicked,is_installed
0,64505.0,0.750943,0.155189
1,64506.0,0.492876,0.328144
2,64507.0,0.471971,0.070001
3,64508.0,0.824673,0.273719
4,64509.0,0.123897,0.180561
...,...,...,...
160968,16240.0,0.573437,0.221137
160969,16241.0,0.791302,0.019241
160970,16242.0,0.490603,0.125397
160971,16243.0,0.967710,0.158026


In [10]:
xgb_output

,RowId,is_clicked,is_installed
0,64505.0,0.307488,0.198798
1,64506.0,0.658150,0.284287
2,64507.0,0.291767,0.077589
3,64508.0,0.958103,0.263996
4,64509.0,0.997934,0.159986
...,...,...,...
160968,16240.0,0.007548,0.218275
160969,16241.0,0.698243,0.006574
160970,16242.0,0.073364,0.015904
160971,16243.0,0.293911,0.126814


In [15]:
num = 2*xgb_output.is_installed.values * cat_output.is_installed.values

In [16]:
denom = xgb_output.is_installed.values + cat_output.is_installed.values

In [17]:
num.max()

1.8255178997523607

In [18]:
denom.max()

1.9107817400222102

In [19]:
f1 = num/denom

In [21]:
f1.min()

0.0007104690133270357

In [22]:
f1

array([0.17430754, 0.30464561, 0.07360014, ..., 0.02822787, 0.14070988,
       0.05517355])

In [23]:
TestResults(row_id=xgb_output.RowId,is_click=np.random.random(f1.shape[0]),
            is_install=f1,model_name="f1_xgb_catb_best",config={'f_b':1})

Saved the test result to csv file as f1_xgb_catb_best_2023-05-27 15:32:56.902811.csv
Saved the model config to json file as f1_xgb_catb_best_2023-05-27 15:32:56.902811.json
